In [1]:
# Connect to OpenEO
import openeo
from openeo.rest.job import JobResults
from openeo.rest.mlmodel import MlModel
import scipy
import numpy as np

connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Authenticated using refresh token.


In [2]:
import json
def read_json(filename: str) -> dict:
    with open(filename) as input:
        field = json.load(input)
    return field

In [3]:
def getBAP(scl, data, reducer="first"):
    mask = (scl == 3) | (scl == 8) | (scl == 9) | (scl == 10)

    # mask is a bit noisy, so we apply smoothening
    # 2D gaussian kernel
    g = scipy.signal.windows.gaussian(11, std=1.6)
    kernel = np.outer(g, g)
    kernel = kernel / kernel.sum()

    # Morphological dilation of mask: convolution + threshold
    mask = mask.apply_kernel(kernel)
    mask = mask > 0.1

    data_masked = data.mask(mask)

    # now select Best Available Pixel based on the mask
    return data_masked.reduce_dimension(reducer=reducer, dimension="t")

In [4]:
# load S2 pre-collection
s2_cube = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=["2020-09-05", "2022-01-01"],
    spatial_extent=read_json("Dataset/training_aoi.geojson"),
    bands=["B02", "B03", "B04", "B08"],
    max_cloud_cover=90,
)

s2_scl = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=["2020-09-05", "2022-01-01"],
    spatial_extent=read_json("Dataset/training_aoi.geojson"),
    bands=["SCL"],
    max_cloud_cover=90,
)


In [ ]:
from openeo.extra.spectral_indices import compute_indices

nbr = compute_indices(s2_cube, indices=["NBR"])

In [ ]:
# Create a Pre-event cloud-free mosaic
rgbnir_pre = getBAP(s2_scl, s2_cube, reducer="median")
nbr_pre = getBAP(s2_scl, nbr, reducer="median")

In [ ]:
rgbnir_nbr = rgbnir_pre.merge_cube(nbr)

In [5]:
# Load Training Data Cube with RGB and NIR Dataset For Training Time Period
# Specific dates S2 images

# yo parena mathi sidai liyesi
# Date of Fire: 05/09/2020 to 20/10/2020; BAP on this date; with median

In [6]:
# Load Training data point Shapefile (Created in GIS Software by random sampling)
# The training polygons are obtained from https://gis.data.ca.gov/ and pre-processed to generate 2000 random samples of fire and non fire area
# This is the specific data (https://gis.data.ca.gov/datasets/CALFIRE-Forestry::recent-large-fire-perimeters-5000-acres-1/explore?location=38.303144%2C-120.691041%2C6.76) and from this dataset only 2019 fire in given aoi area is selected
training_dataset = read_json("Dataset/training_dataset.geojson")

In [7]:
# cube = s2_cube.reduce_dimension(dimension="t", reducer="mean")
# Train the model and DOWNLOAD THE IMAGE USED FOR TRAINING The MODEL
predictors = rgbnir_pre.aggregate_spatial(training_dataset, reducer="median")

In [8]:
model = predictors.fit_class_random_forest(
    target=training_dataset,
)
# Save the model as a batch job result asset
# so that we can load it in another job.
model = model.save_ml_model()

In [9]:
training_job = model.create_job(title="RF bap for WF 300125_test5")
training_job.start_and_wait()

0:00:00 Job 'j-25013014592044639c0bb08993b9058c': send 'start'
0:00:20 Job 'j-25013014592044639c0bb08993b9058c': created (progress 0%)
0:00:25 Job 'j-25013014592044639c0bb08993b9058c': queued (progress 0%)
0:00:32 Job 'j-25013014592044639c0bb08993b9058c': queued (progress 0%)
0:00:40 Job 'j-25013014592044639c0bb08993b9058c': queued (progress 0%)
0:00:51 Job 'j-25013014592044639c0bb08993b9058c': queued (progress 0%)
0:01:03 Job 'j-25013014592044639c0bb08993b9058c': queued (progress 0%)
0:01:20 Job 'j-25013014592044639c0bb08993b9058c': queued (progress 0%)
0:01:39 Job 'j-25013014592044639c0bb08993b9058c': queued (progress 0%)
0:02:04 Job 'j-25013014592044639c0bb08993b9058c': queued (progress 0%)
0:02:34 Job 'j-25013014592044639c0bb08993b9058c': queued (progress 0%)
0:03:11 Job 'j-25013014592044639c0bb08993b9058c': queued (progress 0%)
0:03:58 Job 'j-25013014592044639c0bb08993b9058c': queued (progress 0%)
0:04:57 Job 'j-25013014592044639c0bb08993b9058c': running (progress N/A)
0:05:58 Job

<BatchJob job_id='j-25013014592044639c0bb08993b9058c'>

In [11]:
results = training_job.get_results()
links = results.get_metadata()['links']
ml_model_metadata_url = [link for link in links if 'ml_model_metadata.json' in link['href']][0]['href']
print(ml_model_metadata_url)

https://openeo.dataspace.copernicus.eu/openeo/1.2/jobs/j-25013014592044639c0bb08993b9058c/results/items/M2UyNGUyNTEtMmU5YS00MzhmLTkwYTktZDQ1MDBlNTc2NTc0/02261daddb0890d18c3bf7a4ad2adf62/ml_model_metadata.json?expires=1738862694


In [12]:
# url = "https://openeo.dataspace.copernicus.eu/openeo/1.2/jobs/j-25013014592044639c0bb08993b9058c/results/items/M2UyNGUyNTEtMmU5YS00MzhmLTkwYTktZDQ1MDBlNTc2NTc0/02261daddb0890d18c3bf7a4ad2adf62/ml_model_metadata.json?expires=1738862694"
model = MlModel.load_ml_model(connection=connection, id=ml_model_metadata_url)

In [15]:
# load S2 pre-collection
fire_s2 = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=["2025-01-07", "2025-01-25"],
    spatial_extent=read_json("Dataset/training_aoi.geojson"),
    bands=["B02", "B03", "B04", "B08"],
    max_cloud_cover=90,
)

fire_scl = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=["2025-01-07", "2025-01-25"],
    spatial_extent=read_json("Dataset/training_aoi.geojson"),
    bands=["SCL"],
    max_cloud_cover=90,
)

# Create a Pre-event cloud free mosaic
rgbnir_post = getBAP(fire_scl, fire_s2, reducer="median")


In [17]:
# predict of training data
trained_predicted = rgbnir_pre.predict_random_forest(
    model=model,
    dimension="bands"
)
post_predicted = rgbnir_post.predict_random_forest(
    model=model,
    dimension="bands"
)

In [20]:
trained_predicted.execute_batch(outputfile="trained_predicted.nc", title="trained_predicted")

0:00:00 Job 'j-250130173005459890170bb93210f5ab': send 'start'
0:00:14 Job 'j-250130173005459890170bb93210f5ab': created (progress 0%)
0:00:19 Job 'j-250130173005459890170bb93210f5ab': queued (progress 0%)
0:00:25 Job 'j-250130173005459890170bb93210f5ab': queued (progress 0%)
0:00:33 Job 'j-250130173005459890170bb93210f5ab': queued (progress 0%)
0:00:43 Job 'j-250130173005459890170bb93210f5ab': queued (progress 0%)
0:00:56 Job 'j-250130173005459890170bb93210f5ab': queued (progress 0%)
0:01:11 Job 'j-250130173005459890170bb93210f5ab': running (progress N/A)
0:01:30 Job 'j-250130173005459890170bb93210f5ab': running (progress N/A)
0:01:54 Job 'j-250130173005459890170bb93210f5ab': running (progress N/A)
0:02:24 Job 'j-250130173005459890170bb93210f5ab': running (progress N/A)
0:03:02 Job 'j-250130173005459890170bb93210f5ab': running (progress N/A)
0:03:48 Job 'j-250130173005459890170bb93210f5ab': running (progress N/A)
0:04:47 Job 'j-250130173005459890170bb93210f5ab': running (progress N/A)

<BatchJob job_id='j-250130173005459890170bb93210f5ab'>

In [21]:
post_predicted.execute_batch(outputfile="post_predicted.nc", title="post_predicted")

0:00:00 Job 'j-2501301749044a30ae6d2e2e29711545': send 'start'
0:00:22 Job 'j-2501301749044a30ae6d2e2e29711545': created (progress 0%)
0:00:27 Job 'j-2501301749044a30ae6d2e2e29711545': created (progress 0%)
0:00:33 Job 'j-2501301749044a30ae6d2e2e29711545': created (progress 0%)
0:00:41 Job 'j-2501301749044a30ae6d2e2e29711545': created (progress 0%)
0:00:51 Job 'j-2501301749044a30ae6d2e2e29711545': created (progress 0%)
0:01:04 Job 'j-2501301749044a30ae6d2e2e29711545': created (progress 0%)
0:01:19 Job 'j-2501301749044a30ae6d2e2e29711545': queued (progress 0%)
0:01:39 Job 'j-2501301749044a30ae6d2e2e29711545': queued (progress 0%)
0:02:03 Job 'j-2501301749044a30ae6d2e2e29711545': queued (progress 0%)
0:02:33 Job 'j-2501301749044a30ae6d2e2e29711545': queued (progress 0%)
0:03:11 Job 'j-2501301749044a30ae6d2e2e29711545': running (progress N/A)
0:03:58 Job 'j-2501301749044a30ae6d2e2e29711545': running (progress N/A)
0:04:56 Job 'j-2501301749044a30ae6d2e2e29711545': running (progress N/A)
0:

<BatchJob job_id='j-2501301749044a30ae6d2e2e29711545'>

In [22]:
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
# Create test/dynamic mappping datacube for the same area on 2025 fire
# date of 2025 fire: 07/01/2025--25/01/2025

In [ ]:
# Run the trained model

In [ ]:
# Download the Forest Fire Result

In [19]:
# week = s2cube.aggregate_temporal_period(period="week",reducer="median")
# month = s2cube.aggregate_temporal_period(period="month",reducer="median")

cube1=trained_predicted.save_result("netCDF")
cube2 = post_predicted.save_result("netCDF")

from openeo.internal.graph_building import GraphFlattener
flattener = GraphFlattener()
dict1 = flattener.flatten(cube1.result_node())

#only one node can be marked as 'result':True, so set all others to False
for v in dict1.values():
    v["result"] = False

secondary_result = cube2.result_node()

#by reusing the flattener, the previously flattened nodes are also included
dict2 = flattener.flatten(secondary_result)

final_graph = {"process_graph":dict2}

RuntimeError: Flattening multiple graphs, but not in multi-input mode

In [ ]:
import json
merged = json.dumps(final_graph, indent=2)
print(merged)

connection.create_job(final_graph,title="job with 2 save results")